In [1]:
# Define cutoff date format 'mm/dd/yyyy'
cutoff_date = '11/19/2024'

# Define DM inventory Raw data CSV file name
dm_invent_csv = 'Au_Wire_Nov_21_2024.XLS.csv'

# Define DM Movement Raw data CSV file name
dm_movement_csv = 'movement Jan-Oct_24_120.csv'

In [2]:
import pandas as pd
from datetime import datetime
cutoff_date = datetime.strptime(cutoff_date, '%m/%d/%Y')

# Set the option to display all columns
pd.set_option('display.max_columns', None)

In [3]:
def edit_text_file(input_file, output_file):
    # Read the input file
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Delete rows 1, 2, 3, 4, and 6 (using 0-based index)
    rows_to_delete = [0, 1, 2, 3, 5]  # Convert 1-based to 0-based index
    new_lines = [line for i, line in enumerate(lines) if i not in rows_to_delete]

    # Delete the first character of every remaining row
    new_lines = [line[1:] if len(line) > 1 else line for line in new_lines]

    # Write the modified content to the output file
    with open(output_file, 'w') as file:
        file.writelines(new_lines)

# Specify the input file and output file names
input_file = dm_invent_csv  # Replace with your input file name
output_file = 'dm_invent.csv'

# Call the function to edit the text file
edit_text_file(input_file, output_file)

print(f"File has been edited and saved as {output_file}")

# Import the "data.csv" file into a pandas DataFrame
#df = pd.read_csv(output_file, on_bad_lines='warn')
#df = pd.read_csv(output_file, usecols=[1,18,27])

# Define a custom date parser function
def custom_date_parser(date_str):
    return datetime.strptime(date_str, '%m/%d/%Y')
    
# Import the CSV file with the custom date parser
df = pd.read_csv(output_file,  usecols=[1,8,18,27], parse_dates=['Shlf.Date'], date_parser=custom_date_parser)

# Set new column names using set_axis
df = df.set_axis(['Material_1', 'Batch', 'Shlf_Date', 'TOTAL'], axis='columns')

# Display the DataFrame
#print("\nPandas DataFrame:")
#print(df)
df.info()

File has been edited and saved as dm_invent.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Material_1  10 non-null     object        
 1   Batch       10 non-null     int64         
 2   Shlf_Date   10 non-null     datetime64[ns]
 3   TOTAL       10 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 448.0+ bytes


C:\Users\User\AppData\Local\Temp\ipykernel_18184\3894675093.py:35: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(output_file,  usecols=[1,8,18,27], parse_dates=['Shlf.Date'], date_parser=custom_date_parser)


In [4]:
from pandasql import sqldf

# Define your SQL query
query = """
SELECT Material_1, Shlf_Date, Batch, sum(TOTAL) as TTL
FROM df 
GROUP BY Material_1,Shlf_Date, Batch
ORDER BY Material_1,Shlf_Date, Batch
"""
# Execute the query and create a new DataFrame
df_invent = sqldf(query, globals())

# Display the new DataFrame
print(df_invent)

# Save DataFrame to CSV file
df_invent.to_csv('dm_invent_short.csv', index=False)

  Material_1                   Shlf_Date     Batch    TTL
0     GZ0039  2025-11-11 00:00:00.000000  12356832   27.0
1     GZ0047  2024-10-25 00:00:00.000000  11593813  200.0
2     GZ0047  2025-02-22 00:00:00.000000  11817498  200.0
3     GZ0053  2024-09-04 00:00:00.000000  11467954    1.0
4     GZ0057  2023-06-02 00:00:00.000000  10263087  500.0
5     GZ0057  2023-07-16 00:00:00.000000  10375201    2.0
6     GZ0057  2025-02-13 00:00:00.000000  11793540    1.0
7     GZ0061  2024-10-05 00:00:00.000000  10852739    9.0
8     GZ0061  2025-10-24 00:00:00.000000  12330393    2.0
9     GZ0064  2024-12-18 00:00:00.000000  11691213    4.0


In [5]:
def edit_text_file2(input_file, output_file, encoding='utf-8'):
    # Read the input file with the specified encoding
    with open(input_file, 'r', encoding=encoding) as file:
        lines = file.readlines()

    # Delete rows 1, 2, 3, 4 (using 0-based index)
    rows_to_delete = [0, 1, 2, 3]  # Convert 1-based to 0-based index
    new_lines = [line for i, line in enumerate(lines) if i not in rows_to_delete]

    # Delete the first character of every remaining row
    new_lines = [line[1:] if len(line) > 1 else line for line in new_lines]

    # Write the modified content to the output file with the specified encoding
    with open(output_file, 'w', encoding=encoding) as file:
        file.writelines(new_lines)

# Specify the input file and output file names
input_file = dm_movement_csv  # Replace with your input file name
output_file = 'dm_movement.csv'

# Call the function to edit the text file
edit_text_file2(input_file, output_file)

print(f"File has been edited and saved as {output_file}")

# Read the CSV file, skipping bad lines and displaying warnings
#df2 = pd.read_csv(output_file, on_bad_lines='warn')
df2 = pd.read_csv(output_file, usecols=[2,9,13])

# Remove blank records (rows with any NaN values)
df2_cleaned = df2.dropna()

# Save DataFrame to CSV file
df2_cleaned.to_csv('dm_movement_2-9-13.csv', index=False)

# Import the CSV file with the custom date parser
df2 = pd.read_csv('dm_movement_2-9-13.csv', parse_dates=['Posted on'], date_parser=custom_date_parser)

# Save DataFrame to CSV file
df2.to_csv('dm_movement_2-9-13.csv', index=False)

# Rename the column
# df2 = df2.rename(columns={"Posted on": "Posted_on"})
# df2 = df2.rename(columns={"    Qty in UnE": "Qty_in_UnE"})

# Set new column names using set_axis
df2 = df2.set_axis(['Material', 'Posted_on', 'Qty_in_UnE'], axis='columns')

df2['Posted_on'] = pd.to_datetime(df2['Posted_on'])

# Display the DataFrame
#print("\nPandas DataFrame:")
#print(df2)
df2.info()

File has been edited and saved as dm_movement.csv


C:\Users\User\AppData\Local\Temp\ipykernel_18184\2885995949.py:37: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df2 = pd.read_csv('dm_movement_2-9-13.csv', parse_dates=['Posted on'], date_parser=custom_date_parser)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130537 entries, 0 to 130536
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Material    130537 non-null  object        
 1   Posted_on   130537 non-null  datetime64[ns]
 2   Qty_in_UnE  130537 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.0+ MB


In [6]:
# Define your SQL query
query = """
SELECT Material, strftime('%Y-%m', Posted_on) AS month, SUM(Qty_in_UnE) AS total_value
FROM df2
GROUP BY Material,month
ORDER BY Material,month
"""
# Execute the query and create a new DataFrame
df_movement = sqldf(query, globals())

# Display the new DataFrame
#print(df_movement)

In [7]:
print(df_movement.head(20))

   Material    month  total_value
0    CG0002  2024-01         87.5
1    CG0002  2024-02        157.5
2    CG0002  2024-03        112.5
3    CG0002  2024-04         70.0
4    CG0002  2024-05        102.5
5    CG0002  2024-06         70.0
6    CG0002  2024-07        165.0
7    CG0002  2024-08        162.0
8    CG0002  2024-09        148.0
9    CG0002  2024-10        122.5
10   CZ0134  2024-01          9.0
11   CZ0134  2024-02          8.5
12   CZ0134  2024-03          3.0
13   CZ0134  2024-04         10.0
14   CZ0134  2024-05          4.5
15   CZ0134  2024-07         13.5
16   CZ0134  2024-08          2.2
17   CZ0134  2024-09          3.8
18   CZ0134  2024-10         12.5
19   CZ0135  2024-01        118.0


In [8]:
# Convert DataFrame to pivot table
pivot_table = df_movement.pivot_table(index='Material', columns='month', values='total_value', aggfunc='sum')

# Reset the index to make 'Material' a column again
pivot_table = pivot_table.reset_index()

# Calculate grand total for each material
pivot_table['Grand_Total'] = pivot_table.iloc[:, 1:].sum(axis=1)

# Calculate mean and median for each material
pivot_table['Mean'] = pivot_table.iloc[:, 1:-1].mean(axis=1)
pivot_table['Median'] = pivot_table.iloc[:, 1:-1].median(axis=1)

# Export the result DataFrame to a CSV file
pivot_table.to_csv('dm_movement_pivot.csv', index=False)

print("Pivot table with mean, median, and grand total has been exported to 'dm_movement_pivot.csv'")

Pivot table with mean, median, and grand total has been exported to 'dm_movement_pivot.csv'


In [9]:
print (pivot_table.head(20))

month Material  2024-01  2024-02  2024-03  2024-04  2024-05  2024-06  2024-07  \
0       CG0002     87.5    157.5    112.5     70.0    102.5     70.0    165.0   
1       CZ0134      9.0      8.5      3.0     10.0      4.5      NaN     13.5   
2       CZ0135    118.0    130.0    187.0    112.5    187.5    232.0    208.7   
3       CZ0136     52.5     41.0     47.5     30.0     80.0     85.0    129.0   
4       CZ0138     65.0     92.5     16.0     14.5     58.0     61.0     71.5   
5       CZ0141    735.0   1065.0   1080.0    765.0   1185.0   1335.0   1470.0   
6       CZ0142   1470.0   1335.0   1750.0   1400.0   1620.0   1620.0   1755.0   
7       CZ0143     19.0      4.0      4.5      1.0      NaN      9.5      3.0   
8       CZ0154      6.0      7.5     19.0      9.5      9.0     12.5     21.5   
9       CZ0177    172.5    121.0    173.0     44.5     82.5     75.0     70.0   
10      CZ0245     67.5     62.5     70.0     10.0     46.5     39.0     89.5   
11      CZ0282    635.0    5

In [10]:
# Create DataFrame
df = pd.DataFrame(df_invent)

# Convert Shlf_Date to datetime
df['Shlf_Date'] = pd.to_datetime(df['Shlf_Date'])

# Add Expired_Qty column
df['Expired_Qty'] = df.apply(lambda row: row['TTL'] if row['Shlf_Date'] < cutoff_date else 0, axis=1)

# Display the DataFrame
print(df)

  Material_1  Shlf_Date     Batch    TTL  Expired_Qty
0     GZ0039 2025-11-11  12356832   27.0          0.0
1     GZ0047 2024-10-25  11593813  200.0        200.0
2     GZ0047 2025-02-22  11817498  200.0          0.0
3     GZ0053 2024-09-04  11467954    1.0          1.0
4     GZ0057 2023-06-02  10263087  500.0        500.0
5     GZ0057 2023-07-16  10375201    2.0          2.0
6     GZ0057 2025-02-13  11793540    1.0          0.0
7     GZ0061 2024-10-05  10852739    9.0          9.0
8     GZ0061 2025-10-24  12330393    2.0          0.0
9     GZ0064 2024-12-18  11691213    4.0          0.0


In [11]:
# Define your SQL query
query = """
SELECT Material_1, SUM(TTL) AS OH_TTL, SUM(Expired_Qty) AS Expired
FROM df
GROUP BY Material_1
"""
# Execute the query and create a new DataFrame
df_expired = sqldf(query, globals())

# Display the DataFrame
print(df_expired.head(20))

  Material_1  OH_TTL  Expired
0     GZ0039    27.0      0.0
1     GZ0047   400.0    200.0
2     GZ0053     1.0      1.0
3     GZ0057   503.0    502.0
4     GZ0061    11.0      9.0
5     GZ0064     4.0      0.0


In [12]:
# Define your SQL query
query = """
SELECT *,
    (ex.OH_TTL - ex.Expired) AS Usable
FROM 
    pivot_table AS pt
JOIN 
    df_expired AS ex
ON 
    pt.Material = ex.Material_1;
"""
# Execute the query and create a new DataFrame
df_xxx = sqldf(query, globals())

In [13]:
print(df_xxx.head(20))

  Material  2024-01  2024-02 2024-03 2024-04 2024-05  2024-06 2024-07 2024-08  \
0   GZ0047    200.0      NaN    None    None    None      NaN    None    None   
1   GZ0057      NaN      NaN    None    None    None    500.0    None    None   
2   GZ0064      NaN      6.0    None    None    None      NaN    None    None   

  2024-09 2024-10  Grand_Total   Mean  Median Material_1  OH_TTL  Expired  \
0    None    None        200.0  200.0   200.0     GZ0047   400.0    200.0   
1    None    None        500.0  500.0   500.0     GZ0057   503.0    502.0   
2    None    None          6.0    6.0     6.0     GZ0064     4.0      0.0   

   Usable  
0   200.0  
1     1.0  
2     4.0  


In [14]:
# Export the result DataFrame to a CSV file
df_xxx.to_csv('dm_movement_pivot_w_expired_qty.csv', index=False)

In [15]:
import datetime

# Define the variables
#cutoff_date_str = "Cutoff Date: "+ str(cutoff_date)
#dm_invent_csv = "DM Inventory CSV: dm_inventory.csv"
#dm_movement_csv = "DM Movement CSV: dm_movement.csv"
report_run_date = f"Report Run Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Read the existing content of the file
file_path = 'dm_movement_pivot_w_expired_qty.csv'
with open(file_path, 'r') as file:
    existing_content = file.read()

# Create the new content with the additional lines at the top
new_content = f"Cutoff Date: {cutoff_date}\nDM inventory Raw data CSV file name: \"{dm_invent_csv}\"\nDM Movement Raw data CSV file name: \"{dm_movement_csv}\"\n{report_run_date}\n\n{existing_content}"

# Write the new content back to the file
with open(file_path, 'w') as file:
    file.write(new_content)